In [ ]:
!pip install jellyfish

In [44]:
import glob
import jellyfish
import numpy as np
import pandas as pd

In [49]:
concepts = pd.read_csv('OpenAlex concepts in use (17 August 2022) - concepts.tsv', sep='\t')

In [50]:
concepts_list = concepts['normalized_name'].values
print(len(concepts_list))
concepts_list = concepts_list[~pd.isnull(concepts_list)]
print(len(concepts_list))

65026
65025


In [31]:
files = glob.glob("csv/*.csv") + glob.glob('csv/others/*.csv')
print(files[:4])

['csv/scholars_African Studies.csv', 'csv/scholars_Anthropology.csv', 'csv/scholars_Archaeology.csv', 'csv/scholars_Art and Design Teachers.csv']


In [34]:
fields_github = dict()
for file in files:
    name = file.split('/')[-1]
    if 'scholars_' in name:
        name = name[9:]
    name = name[:-4]
    fields_github[file] = name

In [35]:
fields_github

{'csv/scholars_African Studies.csv': 'African Studies',
 'csv/scholars_Anthropology.csv': 'Anthropology',
 'csv/scholars_Archaeology.csv': 'Archaeology',
 'csv/scholars_Art and Design Teachers.csv': 'Art and Design Teachers',
 'csv/scholars_Asian Studies.csv': 'Asian Studies',
 'csv/scholars_Atmospheric science and air quality.csv': 'Atmospheric science and air quality',
 'csv/scholars_Biblical Studies.csv': 'Biblical Studies',
 'csv/scholars_Bioinformatics.csv': 'Bioinformatics',
 'csv/scholars_Biophysics.csv': 'Biophysics',
 'csv/scholars_Book History (BookHistodons).csv': 'Book History (BookHistodons)',
 'csv/scholars_Chemistry.csv': 'Chemistry',
 'csv/scholars_Communication and Media Studies.csv': 'Communication and Media Studies',
 'csv/scholars_Criminology.csv': 'Criminology',
 'csv/scholars_Crowdsourcing in Cultural Heritage Citizen Science.csv': 'Crowdsourcing in Cultural Heritage Citizen Science',
 'csv/scholars_Dendrochronology.csv': 'Dendrochronology',
 'csv/scholars_Digital

In [51]:
for k,v in fields_github.items():
    print(k)
    dists = []
    for current in concepts_list:
        dists.append(jellyfish.levenshtein_distance(v, current))
    idxs = np.argsort(dists)
    for idx in idxs[:3]:
        print(concepts_list[idx])
    print()

csv/scholars_African Studies.csv
african studies
american studies
asian studies

csv/scholars_Anthropology.csv
anthropology
neurotology
chronology

csv/scholars_Archaeology.csv
archaeology
eschatology
rheology

csv/scholars_Art and Design Teachers.csv
art and design
software design pattern
row and column spaces

csv/scholars_Asian Studies.csv
asian studies
russian studies
urban studies

csv/scholars_Atmospheric science and air quality.csv
atmospheric instability
atmospheric sciences
atmospheric electricity

csv/scholars_Biblical Studies.csv
biblical studies
policy studies
animal studies

csv/scholars_Bioinformatics.csv
bioinformatics
geoinformatics
informatics

csv/scholars_Biophysics.csv
biophysics
geophysics
dinophysis

csv/scholars_Book History (BookHistodons).csv
history of ideas
history of astronomy
history of religions

csv/scholars_Chemistry.csv
chemistry
chemist
geochemistry

csv/scholars_Communication and Media Studies.csv
communication studies
communication disorder
communica

In [52]:
content = open('concepts_unification.txt').read().split('\n\n')
mapconcepts = dict()
for row in content:
    temp = row.split('\n')
    mapconcepts[temp[0]] = tuple(temp[1:])

In [53]:
mapconcepts

{'csv/scholars_African Studies.csv': ('african studies',),
 'csv/scholars_Anthropology.csv': ('anthropology',),
 'csv/scholars_Archaeology.csv': ('archaeology',),
 'csv/scholars_Art and Design Teachers.csv': ('art and design',),
 'csv/scholars_Asian Studies.csv': ('asian studies',),
 'csv/scholars_Atmospheric science and air quality.csv': ('atmospheric sciences',),
 'csv/scholars_Biblical Studies.csv': ('biblical studies',),
 'csv/scholars_Bioinformatics.csv': ('bioinformatics',),
 'csv/scholars_Biophysics.csv': ('biophysics',),
 'csv/scholars_Book History (BookHistodons).csv': ('coding (social sciences) (manual annotation)',),
 'csv/scholars_Chemistry.csv': ('chemistry',),
 'csv/scholars_Communication and Media Studies.csv': ('communication studies',),
 'csv/scholars_Criminology.csv': ('criminology',),
 'csv/scholars_Crowdsourcing in Cultural Heritage Citizen Science.csv': ('crowdsourcing software development',),
 'csv/scholars_Dendrochronology.csv': ('dendrochronology',),
 'csv/schol